This is a kind of 'cheat sheet' for using the module.

## Constructing `Operator`
Operators are the basic building blocks of `Expression`s. 
these inherit `__eq__` from object, so they should *not* be recreated!

In [2]:
from commutation import *
from fractions import Fraction

# Use the constructor
a = Operator('a')
print(a)
print(a.latex_string)

b = Operator('b','S^z_b')
c = Operator('c','S^z_c')
d = Operator('d','S^z_d')
print(b)
print(b.latex_string)

# Operator can also be declared scalar
KA = Operator('KA','K_A',scalar=True)

a
a
b
S^z_b


In [14]:
# making new `a`s is fine, but be warned that doing whis might lead to weird latex rendering

a2 = Operator('a', 'S^z_a')
a3 = Operator('a', 'a')
print(a==a2, a==a3, a2==a3)

True True True


## Constructing `Term`

A `Term` is a product of Operators and "true" scalars, i.e. `int` and `Fraction`.


In [15]:
# A default Term is 1:
print(Term())

+1


In [16]:
# creating them with the constructor:
t = Term(-9,a,b,Fraction(5,6),c,d)
print(t)
print(t.as_latex())

# you can change the rendering of `a` on the fly:
a.latex_string = 'S^z_a'
print(t.as_latex())
show(t)

-15/2 a b c d
-\frac{15}{2} a S^z_b S^z_c S^z_d
-\frac{15}{2} S^z_a S^z_b S^z_c S^z_d


<IPython.core.display.Latex object>

In [17]:
# Terms are naturally built when multiplying Operators:
t = a*b*-9
print(t)


-9 a b


In [18]:
# Copying should be done with the copy constructor
t2 = Term(t)
t2.ops[1] = c
print(t)
print(t2)

-9 a b
-9 a c


In [19]:
### DON'T do this unless you mean to:
# Assignment makes a shallow copy
t2 = t
print(t2)
t2.ops[1] = c
print(t, t2)

-9 a b
-9 a c -9 a c


In [20]:
## Equality of terms is noncommutative:

print(a*b == a*b)
print(a*b == b*a)


True
False


In [21]:
# Terms are searchable:
tt = a*b*b*b*a*b*a*a*c*a*b
print(tt.findall(a*b))

[0, 4, 9]


In [22]:
# Searching avoids collisions
tt = a*b*b*b*b*b*a*a*c*a*b
print(tt.findall(b*b))

[1, 3]


In [23]:
z1 = Operator('z1',r'\zeta_1',scalar=True)
z2 = Operator('z2',r'\zeta_2', scalar=True)
z3 = Operator('z3',r'\zeta_3', scalar=True)

z1_bar = Operator('z1b',r'\zeta_1^*', scalar=True)
z2_bar = Operator('z2b',r'\zeta_2^*', scalar=True)
z3_bar = Operator('z3b',r'\zeta_3^*', scalar=True)

def conjugate(x):
     return x.replaceall((z1, z1_bar), (z2, z2_bar), (z3, z3_bar), (z1_bar, z1), (z2_bar, z2), (z3_bar, z3))
    
xpr = z3*z1_bar*a + z1*z2_bar*b*a
show(xpr)
xprbar=conjugate(xpr)
show(xprbar)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Constructing `Expression`
An `Expression` is a sum of Terms.

In [24]:
# A default `Expression` is 0: (renders as empty space)
print(Expression() + 1)
print(Expression())

  +1



In [25]:
# Expressions can be constructed from multiple arguments:
x = Expression(a*b*b*c,b*c,a*b*c)
show(x)

<IPython.core.display.Latex object>

In [26]:
# or as sums of Terms:
x2 = a*b*c + a*b*b*c + b*c 
show(x2)

<IPython.core.display.Latex object>

In [27]:
# equality is commutative with respect to rearrangements of the sum:
print(x2 == x)

True


In [28]:
# collect() collects like terms:
x = a*b + b*b + Fraction(9/8)*a*b + b*-11*b
x.collect()
show(x)
# and even cancels:
x = 2*a*b + -1*a*b
x.collect()
print(x)


<IPython.core.display.Latex object>

  +1 a b


In [29]:
# Expressions can be substituted into:

tt = a*b*b*b*a*c*a*a*c*a*b

xx = tt + 1
show(xx)

assert xx.terms[0].ops[0] == a

y = xx.substitute(a*b,c+d)
print(xx)
print(y)

# substitute returns a deep copy
y.terms[0].ops[0] = a
print(xx)
print(y)

<IPython.core.display.Latex object>

  +1 a b b b a c a a c a b  +1
  +1 c b b a c a a c c  +1 c b b a c a a c d  +1 d b b a c a a c c  +1 d b b a c a a c d  +1
  +1 a b b b a c a a c a b  +1
  +1 a b b a c a a c c  +1 c b b a c a a c d  +1 d b b a c a a c c  +1 d b b a c a a c d  +1


In [30]:
# multiple substitutions can be chained:
q = Fraction(1,4)

z = xx.sub(b*b, q).sub(a*a,q).sub(c*c, q).sub(a*a,q).sub(b*b, q).sub(c*c,q)
show(z)

<IPython.core.display.Latex object>

In [31]:
# Coefficients of known terms can be pulled out

xx = 7*a*b*b + Fraction(4,5)*a*c*b*b + a*d*b*b + a*c*a*a*b*b + 3*a*b*b
print(xx)

y = xx.coefficient(a*c,'right')
show(y)
y = xx.coefficient(a*c)
show(y)
y = xx.coefficient(b*b)
show(y)


  +7 a b b  +4/5 a c b b  +1 a d b b  +1 a c a a b b  +3 a b b


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [32]:
# Factorisation of leading or trailing terms can be done without specifying which term to look for

fr, ba = xx.factor('right')
print(str(fr) + ' * [' + str(ba) + '  ] = ' +str(fr*ba))

fr, ba = xx.factor()
print('['+str(fr) + '  ] * ' + str(ba) + ' = ' +str(fr*ba))

+1 a * [  +10 b b  +4/5 c b b  +1 d b b  +1 c a a b b  ] =   +10 a b b  +4/5 a c b b  +1 a d b b  +1 a c a a b b
[  +10 a  +4/5 a c  +1 a d  +1 a c a a  ] * +1 b b =   +10 a b b  +4/5 a c b b  +1 a d b b  +1 a c a a b b


## Constructing a CommutatorAlgebra

In [4]:
ca = CommutatorAlgebra()

az = Operator('az','S^z_a')
ap = Operator('a⁺','S^+_a')
am = Operator('a⁻','S^-_a')

# note the funky bracket sequence - set_commutator actually returns a function
ca.set_commutator(az,ap)(ap)
ca.set_commutator(az,am)(-1*am)
ca.set_commutator(ap,am)(2*az)


In [5]:
# there are two modes: move_left and move_right
# Note that pathological commutators will make these fall into recursion loops...

xpr = Expression(az*ap*az*az*am)
ca.move_right(xpr, az)
xpr.collect()
print(xpr) # +1 a⁺ a⁻ az az az  -2 a⁺ a⁻ az az  +1 a⁺ a⁻ az

xpr = Expression(az*ap*az*az*am)
ca.move_left(xpr, am)
xpr.collect()
print(xpr) # +1 a⁻ az a⁺ az az  -2 a⁻ az a⁺ az  +2 az az az az  -1 a⁻ a⁺ az az  +1 a⁻ az a⁺  -4 az az az  +2 a⁻ a⁺ az  +2 az az  -1 a⁻ a⁺

# these will warn you if you add an unknown operator...
xpr2 = Expression(az*ap*am*c*am)
ca.move_right(xpr2, az)
xpr2.collect()
print(xpr2) #   +1 a⁺ a⁻ c a⁻ az  -1 a⁺ a⁻ c a⁻

# # ... but scalars are fine.
xpr3 = Expression(az*ap*am*KA*am)
ca.move_right(xpr3, az)
xpr3.collect()
print(xpr3) #   +1 a⁺ a⁻ KA a⁻ az  -1 a⁺ a⁻ KA a⁻

  +1 a⁺ a⁻ az az az  -2 a⁺ a⁻ az az  +1 a⁺ a⁻ az
  +1 a⁻ az a⁺ az az  -2 a⁻ az a⁺ az  +2 az az az az  -1 a⁻ a⁺ az az  +1 a⁻ az a⁺  -4 az az az  +2 a⁻ a⁺ az  +2 az az  -1 a⁻ a⁺
  +1 a⁺ a⁻ c a⁻ az  -1 a⁺ a⁻ c a⁻
  +1 a⁺ a⁻ KA a⁻ az  -1 a⁺ a⁻ KA a⁻


/opt/homebrew/lib/python3.10/site-packages/commutation/commutatoralgebra.py:72: UserWarning: Non-scalar operator "c" is not in the commutator database, assuming it commutes...
  warn(s)


In [35]:
x = ap*az*am + am*az*ap

ca.move_right(x,am)
ca.move_right(x,ap)
print(x)
#+2 az a⁻ a⁺  +2 az az  -2 az

y = 4*ap*am - 4*ap*am + 1
print(y)
y.collect()
print(y)


  +2 az a⁻ a⁺  +2 az az  -2 az
  +4 a⁺ a⁻  -4 a⁺ a⁻  +1
  +1


In [36]:
# Scalar quantities can be bubbled to the front more efficiently:
x = a*b*KA*c*KA + KA*b*KA
x.move_scalars()
show(x)


# this also works for Term:
trm = a*b*KA*c*KA
print(trm)
trm.move_scalars()
print(trm)

<IPython.core.display.Latex object>

+1 a b KA c KA
+1 KA KA a b c


In [37]:
# just a check
xpr = Expression(az*ap*az*az*am)
ca.move_right(xpr, az)
ca.move_left(xpr, az)
xpr.collect()
show(xpr)

xpr = Expression(az*ap*az*az*am)
# ca.move_right(xpr, az)
ca.move_left(xpr, az)
ca.move_left(xpr, az)
xpr.collect()
show(xpr)

# answers are the same. Phew!

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [38]:
a = Operator('a')
b = Operator('b')
c = Operator('c')
d = Operator('d')
e = Operator('e')
f = Operator('f')

cb = CommutatorAlgebra()

cb.set_commutator(a,b)(Fraction(-7,2)*(a+b))
cb.set_commutator(a,c)(Fraction(11,3)*(a+b*d))
cb.set_commutator(a,d)(Fraction(1,5)*(d))
cb.set_commutator(b,c)(Fraction(11,5)*(a*b + c*a) )

x = a*b*c*d + 1
cb.move_right(x,a)
show(x)

x = a*b*c*d + 1
cb.move_right(x,b)
show(x)

x = d*c*b*a + 1
cb.move_left(x,b)
show(x)




<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [39]:
# pathological
cb = CommutatorAlgebra()

cb.set_commutator(a,b)(a+a*c)
cb.set_commutator(c,a)(a)

x = a*c*a*b + 0
cb.move_right(x,a)
show(x)
x.collect()
print(x)


<IPython.core.display.Latex object>

  +1 c b a a  -3 c a a  +2 c c a a  -1 b a a  +1 a a


## Working with `Expression`

In [40]:
f = Operator('f','S^z_f')

x = a*b*c*d*f + a*b*b*d*f
fr, ba = x.factor()
print(f"x = ({fr}) * ({ba}) ")
# (   +1 a b c  +1 a b b )*( +1 d f )

fr, ba = x.factor('left')
print(f"x = ({fr}) * ({ba}) ")
# (   +1 a b c  +1 a b b )*( +1 d f )

fr, ba = x.factor('right')
print(f"x = ({fr}) * ({ba}) ")
# ( +1 a b )*(   +1 c d f  +1 b d f )

x = (  +1 a b c  +1 a b b) * (+1 d f) 
x = (  +1 a b c  +1 a b b) * (+1 d f) 
x = (+1 a b) * (  +1 c d f  +1 b d f) 


In [41]:
x = a*b + b*c*c
y = x.replaceall((b,a),(a,b))
print(y)

  +1 b a  +1 a c c
